In [7]:
import torch
import torch.nn as nn
import torchvision.models as models

import string
import dataset
import config
import data_train
import config
import model
import os

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import CTCLoss
from tqdm import tqdm
import config

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Mapping index → char (ubah sesuai datasetmu)
idx2char = {i: chr(96+i) for i in range(1, 27)}  # 1='a' ... 26='z'
idx2char[0] = "-"  # blank untuk CTC


def decode_prediction(logits):
    """Greedy decode dari output CTC"""
    out = torch.argmax(logits, dim=2)  # [T, B]
    out = out.permute(1, 0)  # [B, T]
    results = []
    for seq in out:
        prev = -1
        s = ""
        for idx in seq.cpu().numpy():
            if idx != prev and idx != 0:  # remove repeat & blank
                s += idx2char.get(idx, "?")
            prev = idx
        results.append(s)
    return results


def train_one_epoch(model, train_loader, optimizer, criterion, epoch, print_every=100):
    model.train()
    train_loss = 0.0
    pbar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f"Epoch {epoch}")

    for batch_idx, batch in pbar:
        images = batch["images"].to(device)           # [B, C, H, W]
        targets = batch["targets"].to(device)         # [B, max_len]
        targets_lengths = batch["targets_lengths"]    # [B]

        batch_size = images.size(0)

        # Forward
        logits = model(images)   # [T, B, num_classes]
        log_probs = F.log_softmax(logits, dim=2)

        input_lengths = torch.full(size=(batch_size,), fill_value=logits.size(0), dtype=torch.long).to(device)

        # Hitung CTC Loss
        loss = criterion(log_probs, targets, input_lengths, targets_lengths)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Print sample prediction tiap 10 batch
        if batch_idx % print_every == 0:
            decoded = decode_prediction(log_probs.detach())
            target_strs = []
            for t in targets:
                s = "".join([idx2char[i.item()] for i in t if i.item() != 0])
                target_strs.append(s)

            print(f"\nBatch {batch_idx} - Pred: {decoded[0]} | GT: {target_strs[0]}")

        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    return train_loss / len(train_loader)


def validate(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            images = batch["images"].to(device)
            targets = batch["targets"].to(device)
            targets_lengths = batch["targets_lengths"]

            batch_size = images.size(0)
            logits = model(images)
            log_probs = F.log_softmax(logits, dim=2)
            input_lengths = torch.full(size=(batch_size,), fill_value=logits.size(0), dtype=torch.long).to(device)

            loss = criterion(log_probs, targets, input_lengths, targets_lengths)
            val_loss += loss.item()

    return val_loss / len(val_loader)


def train_model(
    train_loader, 
    val_loader, 
    num_classes=27, 
    hidden_state=128, 
    epochs=10, 
    lr=1e-3, 
    checkpoint_path="checkpoint.pth"
):
    # Init model
    net = model.model_wordRec(num_classes=num_classes, hidden_state=hidden_state).to(device)
    criterion = CTCLoss(blank=0, reduction="mean", zero_infinity=True)
    optimizer = optim.Adam(net.parameters(), lr=lr)

    start_epoch = 1

    # Cek kalau ada checkpoint, resume
    if os.path.exists(checkpoint_path):
        print(f"🔄 Loading checkpoint from {checkpoint_path} ...")
        checkpoint = torch.load(checkpoint_path, map_location=device)
        net.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"Resumed from epoch {checkpoint['epoch']}")

    for epoch in range(start_epoch, epochs+1):
        train_loss = train_one_epoch(net, train_loader, optimizer, criterion, epoch)
        val_loss = validate(net, val_loader, criterion)

        print(f"Epoch [{epoch}/{epochs}] - Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

        # ✅ Simpan checkpoint tiap epoch
        torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss': val_loss,
        }, checkpoint_path)
        print(f"💾 Checkpoint saved at epoch {epoch}")

    return net




Using device: cuda


In [9]:
print(torch.cuda.is_available())  
print(torch.cuda.get_device_name(0)) 


True
NVIDIA GeForce RTX 3050 6GB Laptop GPU


In [10]:
model_trained = train_model(
    data_train.train_loader, 
    data_train.val_loader, 
    num_classes=27,   # 26 huruf + blank
    epochs=config.EPOCHS, 
    lr=0.001
)


Epoch 1:   0%|          | 1/2000 [00:01<27:09,  1.23it/s, loss=22.2492]


Batch 0 - Pred: wvxit | GT: dktegqe


Epoch 1:   5%|▌         | 101/2000 [00:22<06:54,  4.58it/s, loss=3.5814]


Batch 100 - Pred:  | GT: pcikhdn


Epoch 1:  10%|█         | 201/2000 [00:44<06:32,  4.59it/s, loss=3.6157]

...


Batch 300 - Pred: axvmgf | GT: axvmgf


Epoch 3:  20%|██        | 401/2000 [01:28<06:05,  4.38it/s, loss=0.0957]


Batch 400 - Pred: lbihyfpemk | GT: lbihyfpemk


Epoch 3:  23%|██▎       | 452/2000 [01:40<05:43,  4.51it/s, loss=0.0051]


KeyboardInterrupt: 